# IMDb Data Merging & Manipulation

Author: Oliver Gladfelter

Date: 4/8/18

Objective: Merging multiple IMDb data tables together, collect information - including the names of the writing staff - for each episode in the top US comedies of the 21st century

In [2]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
# the crew table gives the title ID and the writer ID 
crew = pd.read_csv('data_1.tsv', sep = '\t')

# the names table gives writer ID and writer name
names = pd.read_csv('data_2.tsv', sep = '\t')

# contains tconst, titleType, startYear, endYear, genres
basicInfo = pd.read_csv('basicTitle.tsv', sep = '\t')

# contains ratings info
ratings = pd.read_csv('ratings.tsv', sep = '\t')

# info about episodes
episodeInfo = pd.read_csv('data.tsv', sep = '\t')

# matches IDs with title names
akas = pd.read_csv('akas.tsv', sep = '\t')

In [4]:
def convertToInt(value):
    if value != "\\N":
        return int(value)

basicInfo['startYear'] = basicInfo['startYear'].apply(convertToInt)

In [5]:
# subset basics into a data frame including only tv episodes from the comedy genre and released 2000 or later
episodes = basicInfo[(basicInfo['titleType'] == 'tvEpisode') & 
                     (basicInfo['startYear'] > 1999) & (basicInfo['genres'] == 'Comedy')]

In [6]:
# merging it all together and subsetting to include only US episodes
episodesRatings = episodes.merge(ratings)

episodesRatingsSeries = episodesRatings.merge(episodeInfo)

episodesRatingsSeriesTitle = episodesRatingsSeries.merge(akas, how='inner', left_on = 'parentTconst', right_on = 'titleId')

# subset to include only tv episodes produced in the US --> CAN THIS BE DONE EARLIER? WHAT TABLE DOES 'region' COME FROM?
episodesRatingsSeriesTitleUS = episodesRatingsSeriesTitle[episodesRatingsSeriesTitle['region'] == 'US']

episodesRatingsSeriesTitleUS = episodesRatingsSeriesTitleUS.drop(['titleId', 'ordering', 'endYear', 'originalTitle', 'titleType', 'isAdult', 'genres', 'region', 'language', 'types', 'isOriginalTitle', 'attributes'], axis = 1)

In [168]:
episodesRatingsSeriesTitleUS.head()

,tconst,primaryTitle,startYear,runtimeMinutes,averageRating,numVotes,parentTconst,seasonNumber,episodeNumber,title
43,tt0366946,Austin Powers in Goldmember,2002.0,30,6.1,14,tt0353055,\N,\N,Comedy Central Canned Ham
44,tt0366949,Death to Smoochy,2002.0,30,5.8,20,tt0353055,\N,\N,Comedy Central Canned Ham
45,tt0366951,Mr. Deeds,2002.0,30,6.7,15,tt0353055,\N,\N,Comedy Central Canned Ham
46,tt0366953,Stealing Harvard,2002.0,30,6.3,9,tt0353055,\N,\N,Comedy Central Canned Ham
47,tt0366954,Super Troopers,2002.0,30,6.1,74,tt0353055,\N,\N,Comedy Central Canned Ham


In [103]:
# find the TV shows in the dataframe with the highest average number of votes per episode

# group by the series title, producing a table where each row is a unique TV series
# and its mean number of ratings per episode
seriesRatings = episodesRatingsSeriesTitleUS.groupby(['title']).mean()

# delete unnecessary columns
seriesRatings = seriesRatings.drop(['ordering', 'averageRating', 'startYear'], axis=1)

seriesRatingsOrder = seriesRatings.sort_values(by = 'numVotes')

# save just the top 25 
topComedies = seriesRatingsOrder.tail(21)
topComedies = topComedies.reset_index()

In [105]:
topComedies

,title,numVotes
0,The Girlie Show,554.919708
1,Untitled Tina Fey Project,554.919708
2,30 Rock,554.919708
3,Chicks and Dicks,590.760870
4,New Girl,590.760870
5,Curb Your Enthusiasm,687.077778
6,Jean-Claude Van Johnson,926.833333
7,Silicon Valley,1044.948718
8,The Untitled Greg Daniels/Mike Schur/Amy Poehl...,1048.520000
9,Parks and Rec,1048.520000


In [169]:
# Subset the episodesRatingsSeriesTitleUS dataframe to include only the top 12 unique TV series
topComediesData = episodesRatingsSeriesTitleUS[(episodesRatingsSeriesTitleUS['title'] == 'Community') | (episodesRatingsSeriesTitleUS['title'] == 'The Office') | (episodesRatingsSeriesTitleUS['title'] == 'Master of None') | (episodesRatingsSeriesTitleUS['title'] == 'Jerks') | (episodesRatingsSeriesTitleUS['title'] == 'Rick And Morty') | (episodesRatingsSeriesTitleUS['title'] == "It's Always Sunny in Philadelphia") | (episodesRatingsSeriesTitleUS['title'] == 'Arrested Development') | (episodesRatingsSeriesTitleUS['title'] == 'Parks and Rec') | (episodesRatingsSeriesTitleUS['title'] == 'Silicon Valley') | (episodesRatingsSeriesTitleUS['title'] == 'Jean-Claude Van Johnson') | (episodesRatingsSeriesTitleUS['title'] == 'Curb Your Enthusiasm') | (episodesRatingsSeriesTitleUS['title'] == 'New Girl') | (episodesRatingsSeriesTitleUS['title'] == '30 Rock')]

In [170]:
topComediesData.head()

,tconst,primaryTitle,startYear,runtimeMinutes,averageRating,numVotes,parentTconst,seasonNumber,episodeNumber,title
1389,tt0515207,Afternoon Delight,2004.0,30,9.1,1603,tt0367279,2,6,Arrested Development
1414,tt0515208,Altar Egos,2004.0,22,8.6,1439,tt0367279,1,16,Arrested Development
1439,tt0515209,Amigos,2004.0,30,8.9,1444,tt0367279,2,3,Arrested Development
1464,tt0515210,Beef Consomme,2004.0,22,8.5,1463,tt0367279,1,13,Arrested Development
1489,tt0515211,Best Man for the Gob,2004.0,22,8.3,1364,tt0367279,1,19,Arrested Development


In [143]:
episodesCrew = topComediesData.merge(crew)

In [144]:
# in the crews table, multiple writer IDs of one project are included together in one
# long string, which makes a merge with the names table successful only for projects
# with only one writer. This for loop munges the crew table into a table where
# projects with multiple writers are split into multiple rows, one for each writer

writingStaff = []

for index, row in episodesCrew.iterrows():
    
    if row['writers'] != r'\N':
        episodesOrMovies = row['tconst']
        writersP = row['writers']
        
        for writer in writersP.split(','):
            newRow = [episodesOrMovies, writer]
            writingStaff.append(newRow)

writingStaffDF = pd.DataFrame(writingStaff, columns = ['tconst', 'nconst'])

In [145]:
# Merge with the names table to add the primaryName column, which gives the writer name associated with a writer ID
writingStaffNames = writingStaffDF.merge(names)

In [146]:
episodeWriters = writingStaffNames.drop(['nconst', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], axis=1)

In [148]:
# add a new column of blank strings
episodeWriters['writers'] = ''

In [ ]:
# iterate over the length of the data frame in order to create full lists of writers involved, rather
# than having writing staffs seperated over multiple rows
for episode in range(0, len(episodeWriters)):
    
    # create a variable holding the string of the writers name for current row
    names = episodeWriters['primaryName'][episode]
    count = 1
    
    # while subsequent rows contain information about the current row's same movie, add the writer's
    # names to the 'names' variable. 
    while episodeWriters['tconst'][episode] == episodeWriters['tconst'][episode + count]:
        names = names + ", " + episodeWriters['primaryName'][episode + count]
        count = count + 1
        
    # once the last row for the given movie is reached and all writers have been added to the 
    # 'names' variable, replace the empty string in the 'writers' column with the string held by 'names'
    episodeWriters['writers'][episode] = names

In [163]:
# drop duplicate movies in the data frame, always keeping the first instance of each repeated movie
# because the full writing staffs are only included in the first instance
episodeWriters = episodeWriters.drop_duplicates(subset = 'tconst', keep = 'first')

del episodeWriters['primaryName']

episodeWriters.head()

,tconst,writers
0,tt0515207,Mitchell Hurwitz
1,tt0515208,Mitchell Hurwitz
2,tt0515209,Mitchell Hurwitz
3,tt0515210,Mitchell Hurwitz
4,tt0515211,Mitchell Hurwitz


In [157]:
topComediesDataWithWriters = topComediesData.merge(episodeWriters)

In [159]:
topComediesDataWithWriters.head()

,tconst,primaryTitle,startYear,runtimeMinutes,averageRating,numVotes,parentTconst,seasonNumber,episodeNumber,title,writers
0,tt0515207,Afternoon Delight,2004.0,30,9.1,1603,tt0367279,2,6,Arrested Development,Mitchell Hurwitz
1,tt0515208,Altar Egos,2004.0,22,8.6,1439,tt0367279,1,16,Arrested Development,Mitchell Hurwitz
2,tt0515209,Amigos,2004.0,30,8.9,1444,tt0367279,2,3,Arrested Development,Mitchell Hurwitz
3,tt0515210,Beef Consomme,2004.0,22,8.5,1463,tt0367279,1,13,Arrested Development,Mitchell Hurwitz
4,tt0515211,Best Man for the Gob,2004.0,22,8.3,1364,tt0367279,1,19,Arrested Development,Mitchell Hurwitz


In [162]:
episodesWithWriters.to_csv("C:\\Users\\Oliver\\Documents\\finalEpisodeData.csv")